In [8]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [28]:
df = pd.read_csv('pred_2021_srs.csv')
df['pred_spread'] = df['pred_spread']*-1
df
df1 = df.copy()

In [44]:
def odds_to_prob(x):
    if x < 0:
        return round(abs(x) / (100+abs(x)), 4)
    else:
        return round(100 / (100+abs(x)), 4)
    
def prob_to_spread(prob):
    sigma = 14.4
    spread = round(norm.ppf(prob)*sigma, 2)
    if prob > 0.5:
        return -1*spread
    else:
        return spread
    
def spread_winprob(diff):
    sigma = 14.4
    away_spread_winprob = round(norm.cdf(diff/sigma), 4)
    home_spread_winprob = 1 - away_spread_winprob
#     return max(away_spread_winprob, home_spread_winprob)
    return home_spread_winprob

def bet(home_winprob):
    r = 10/11
    p = max(home_winprob, 1 - home_winprob)
    if p < 0.5238:
        return 0
    kelly = (p*r + p - 1)/r
    frac = 0.25
    
    # FADE MYSELF???
    if home_winprob > 0.5:
        return frac*kelly
    else:
        return -1*frac*kelly

In [45]:
def process(df):
    df = df.copy()
    df['margin'] = df['final_v'] - df['final_h']
    # df['vegas_home_win_prob'] = df['ml_h'].apply(odds_to_prob)
    # df['vegas_away_win_prob'] = df['ml_v'].apply(odds_to_prob)
    # df['home_rating'] = home_ratings
    # df['away_rating'] = away_ratings
    # df['home_win_prob'] = [round(prob, 4) for prob in home_winprobs]
    # df['away_win_prob'] = 1 - df['home_win_prob']
    
#     df['pred_spread'] = df['hw_prob'].apply(prob_to_spread)
    df['spread_diff_vegas'] = df['pred_spread'] - df['spread_close']
    df['vegas_spread_winprob'] = df['spread_diff_vegas'].apply(spread_winprob)
    df['bet_size'] = df['vegas_spread_winprob'].apply(bet)
    df['pred_error'] = df['margin'] - df['pred_spread']
    df['spread_outcome'] = df['pred_error'].apply(lambda x: -1 if x < 0 else 1)
    df['bet_outcome'] = (df['bet_size'] * df['spread_outcome']).apply(lambda x: 1 if x > 0 else 0)
    return df

In [46]:
process(df)

,Unnamed: 0,id,Date,v,final_v,ml_v,h,final_h,ml_h,total_open,total_close,spread_open,spread_close,v_win_prob,h_win_prob,actual_spread,pred_spread,hw_prob,margin,spread_diff_vegas,vegas_spread_winprob,bet_size,pred_error,spread_outcome,bet_outcome
0,0,0,1222,GoldenState,99,265,Brooklyn,125,-320,228.0,234.5,-5.5,-7.5,0.264484,0.735516,26,-5.150453,0.638781,-26,2.349547,0.4352,-0.021520,-20.849547,-1,1
1,1,1,1222,LAClippers,116,115,LALakers,109,-135,223.0,219.0,-2.5,-2.0,0.447406,0.552594,-7,-1.713414,0.547032,7,0.286586,0.4921,0.000000,8.713414,1,0
2,2,2,1223,Charlotte,114,-140,Cleveland,121,120,215.0,217.0,-1.0,3.0,0.562044,0.437956,7,0.985945,0.472894,-7,-2.014055,0.5556,0.016690,-7.985945,-1,0
3,3,3,1223,NewYork,107,270,Indiana,121,-330,214.5,215.5,-8.5,-7.5,0.260448,0.739552,14,-10.169973,0.758466,-14,-2.669973,0.5735,0.026088,-3.830027,-1,0
4,4,4,1223,Miami,107,-190,Orlando,113,170,220.5,219.0,3.5,4.5,0.638854,0.361146,6,4.998876,0.365141,-6,0.498876,0.4862,0.000000,-10.998876,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,1166,1166,708,Milwaukee,108,160,Phoenix,118,-180,221.0,221.0,-5.5,-4.5,0.374332,0.625668,10,3.886682,0.394332,-10,8.386682,0.2801,-0.102947,-13.886682,-1,1
1167,1167,1167,711,Phoenix,100,180,Milwaukee,120,-210,222.0,220.0,-3.5,-4.5,0.345212,0.654788,20,-6.238518,0.666490,-20,-1.738518,0.5480,0.012700,-13.761482,-1,0
1168,1168,1168,714,Phoenix,103,170,Milwaukee,109,-190,220.5,221.0,-3.5,-4.5,0.361146,0.638854,6,-6.238518,0.666490,-6,-1.738518,0.5480,0.012700,0.238518,1,1
1169,1169,1169,717,Milwaukee,123,150,Phoenix,119,-170,218.5,220.0,-4.0,-4.0,0.388489,0.611511,-4,3.886682,0.394332,4,7.886682,0.2920,-0.096700,0.113318,1,0


In [47]:
process(df)['bet_outcome'].mean()

0.5610589239965841

In [38]:
process(df).to_csv('nba_betting_srs_2020-21.csv')